In [ ]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.7/535.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.9 MB/s eta 0:00:00


In [ ]:
# Including required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,precision_score,recall_score, f1_score, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
import optuna
import shap

In [ ]:
df=pd.read_csv('Fraud.csv')

In [ ]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231025 entries, 0 to 231024
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   step            231025 non-null  int64  
 1   type            231025 non-null  object 
 2   amount          231025 non-null  float64
 3   nameOrig        231025 non-null  object 
 4   oldbalanceOrg   231025 non-null  float64
 5   newbalanceOrig  231025 non-null  float64
 6   nameDest        231024 non-null  object 
 7   oldbalanceDest  231024 non-null  float64
 8   newbalanceDest  231024 non-null  float64
 9   isFraud         231024 non-null  float64
 10  isFlaggedFraud  231024 non-null  float64
dtypes: float64(7), int64(1), object(3)
memory usage: 19.4+ MB


In [ ]:
df['type'].unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

#Preprocessing


In [ ]:
# Filling null values(missing), if any present in the dataset
df.fillna(method='ffill',inplace=True)

In [ ]:
# Outlier detection and removal using IQR
Q1=df['amount'].quantile(0.25)
Q3=df['amount'].quantile(0.75)
IQR=Q3-Q1
df=df[~((df['amount'] < (Q1 - 1.5*IQR))| (df['amount'] > (Q3+1.5*IQR)))]

In [ ]:
# Selecting the variables to be included in the model
numerical_cols=['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']
categorical_cols=['type']

In [ ]:
# Multi-collinearity
def calculate_vif(df):
    X=df.copy()
    X=add_constant(X)
    vif=pd.DataFrame()
    vif["variables"]=X.columns
    vif["VIF"]=[variance_inflation_factor(X.values,i) for i in range(X.shape[1])]
    return vif

In [ ]:
result=calculate_vif(df[numerical_cols])

In [ ]:
print("Id", result)

Id         variables         VIF
0           const    1.887314
1          amount    1.186487
2   oldbalanceOrg  851.386395
3  newbalanceOrig  853.430367
4  oldbalanceDest   14.621740
5  newbalanceDest   14.879872


In [ ]:
# Handling categorical variables with LabelEncoder
l = LabelEncoder()
df['type'] = l.fit_transform(df['type'])

In [ ]:
# Preprocessing the numerical data
numerical_trans=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])

In [ ]:
# Preprocessing categorical columns
categorical_trans=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
# Bundling the numerical and categorical data
preprocessor=ColumnTransformer(
    transformers=[
('num',numerical_trans,numerical_cols),
('cat',categorical_trans,categorical_cols)]
)


#MODEL

I have used Random Forest Classfier (RFC) for Model training, It is an ensemble learning method that combanies multiple decision trees to create an accuarate model.


**How it works?**
- A set of decision trees is built using random subsets of the data (bootstrapping).
- Each tree is trained on a different subset of features (random feature selection).
- The final prediction is made by aggregating the predictions of all individual trees (voting or averaging).

In [ ]:
model=RandomForestClassifier(random_state=42)

In [ ]:
# Bundling the preprocessing and model in the pipeline
clf=Pipeline(steps=[('preprocessor',preprocessor),
                   ('model',model)])

In [ ]:
# Feature Selection and data splitting
X=df.drop(['isFraud','nameOrig','nameDest','isFlaggedFraud','step'],axis=1)
y=df['isFraud']
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


Hyperparameter Tuning

- Hyperparameters are parameters that are set before training the model. They control various aspects of the model’s behavior, such as tree depth, number of trees, and minimum samples per leaf.
- Optuna is a Python library for automated hyperparameter optimization. It uses Bayesian optimization to efficiently explore the hyperparameter space and find the best configuration.

Here’s how you it can be used for hyperparameter tuning with an RFC:
- Define an objective function that takes hyperparameters as input.
 - Inside the objective function:
   - Create an RFC model with the specified hyperparameters.
   - Train the model on your dataset.
   - Evaluate its performance using cross-validation or a validation set.
 - Optuna will iteratively explore different hyperparameter combinations, aiming to minimize/maximize the objective function (e.g., accuracy, AUC-ROC, etc.).
 - After a predefined number of trials, Optuna selects the best hyperparameters.


In [ ]:
# Hyperparameter tuning using Optuna
study = optuna.create_study(direction='maximize')

[I 2024-02-28 08:02:33,708] A new study created in memory with name: no-name-9f4b15b3-06f4-4063-a4bd-ffc1887c7abc


When selecting variables (features) to be included in a Random Forest Classifier (RFC) model, several considerations come into play. Let's explore how this process typically works:
1.	Feature Importance:
-	RFC inherently evaluates feature importance during its construction.
-	Here's how it works:
 -	During training, each decision tree in the forest is built using a random subset of features.
 -	The algorithm assesses the importance of each feature by measuring how much it contributes to reducing impurity (e.g., Gini impurity or information gain) across all trees.
 -	Features that consistently lead to better splits are considered more important.
-	You can access feature importances using the feature_importances_ attribute after training the RFC model.
2.	Practical Steps for Feature Selection:
-	While RFC automatically considers feature importance, you can further refine your feature set:
 -	Top-k Features: Select the top-k most important features based on their importance scores.
 -	Threshold: Set a threshold (e.g., 0.01) and include features with importance scores above that threshold.
 -	Domain Knowledge: Leverage domain expertise to choose relevant features.
 -	Recursive Feature Elimination (RFE): Iteratively remove less important features.
 -	SelectKBest: Use statistical tests (e.g., chi-squared, ANOVA) to select the k best features.
 -	L1 Regularization (Lasso): Penalize less important features during model training.



In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_categorical('max_depth', [None, 10, 20, 30])
    min_samples_leaf = trial.suggest_categorical('min_samples_leaf', [1, 2, 4])

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

    clf = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    auc = cross_val_score(clf, x_train, y_train, cv=3, scoring='roc_auc').mean()

    return auc

In [ ]:
study.optimize(objective, n_trials=10)

[I 2024-02-28 08:03:19,103] Trial 0 finished with value: 0.8891226286253904 and parameters: {'n_estimators': 129, 'max_depth': 20, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.8891226286253904.
[I 2024-02-28 08:04:01,135] Trial 1 finished with value: 0.9713714222920133 and parameters: {'n_estimators': 134, 'max_depth': 30, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.9713714222920133.
[I 2024-02-28 08:04:49,507] Trial 2 finished with value: 0.8893905194299966 and parameters: {'n_estimators': 157, 'max_depth': 30, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.9713714222920133.
[I 2024-02-28 08:05:27,379] Trial 3 finished with value: 0.9626429074366766 and parameters: {'n_estimators': 123, 'max_depth': None, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.9713714222920133.
[I 2024-02-28 08:07:07,983] Trial 5 finished with value: 0.9709971653873343 and parameters: {'n_estimators': 193, 'max_depth': 20, 'min_samples_leaf': 4}. Best is trial 4 with value: 0.9

In [ ]:
# Get the best hyperparameters
best_params = study.best_params
best_model_optuna = RandomForestClassifier(**best_params, random_state=42)

In [ ]:
# Fit the best model on the entire dataset
best_model_optuna.fit(x_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=2, n_estimators=137,
                       random_state=42)

In [ ]:
# Fitting the model on training data
clf.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['amount', 'oldbalanceOrg',
                                                   'newbalanceOrig',
                                                   'oldbalanceDest',
                                                   'newbalanceDest']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['type'])])),
                ('model', RandomForestClassifier(random_state=42))])

In [ ]:
x_test

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
185961,1,94693.38,0.00,0.00,1437424.12,1532117.50
123397,3,1887.09,156660.00,154772.91,0.00,0.00
132186,3,24429.38,450446.48,426017.10,0.00,0.00
224137,1,85348.28,0.00,0.00,5780760.16,7383414.36
115261,3,11865.40,209.00,0.00,0.00,0.00
...,...,...,...,...,...,...
23352,0,165729.45,52810.96,218540.41,318544.40,191289.18
64839,1,192239.75,806.00,0.00,0.00,339300.49
81976,3,22892.58,0.00,0.00,0.00,0.00
139725,1,86743.27,0.00,0.00,2065123.35,2151866.62


In [ ]:
# Make predictions on the test set
y_pred = clf.predict(x_test)
print(y_pred)

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
sample_size = 5
sample_test_set = pd.DataFrame({
    'step': np.random.randint(1, 20, size=sample_size),
    'type': np.random.randint(1, 5, size=sample_size),
    'amount': np.random.uniform(1000, 100000, size=sample_size),
    'oldbalanceOrg': np.random.uniform(0, 50000, size=sample_size),
    'newbalanceOrig': np.random.uniform(0, 50000, size=sample_size),
    'oldbalanceDest': np.random.uniform(0, 200000, size=sample_size),
    'newbalanceDest': np.random.uniform(0, 200000, size=sample_size),
    'isFlaggedFraud': np.random.choice([0.0, 1.0], size=sample_size)
})

In [ ]:
# Create a DataFrame with a specific fraud case
sample_data = pd.DataFrame({
    'type': [1],
    'amount': [150000],
    'oldbalanceOrg': [50000],
    'newbalanceOrig': [0],
    'oldbalanceDest': [200000],
    'newbalanceDest': [350000],

})

In [ ]:
# Predicting for a specific case
y_pred_one = best_model_optuna.predict(sample_data)
print(np.round(y_pred_one))

[0.]


In [ ]:
# Feature Importance Analysis
importances= best_model_optuna.feature_importances_
feature_importances=pd.Series(importances, index=x_train.columns)
key_factors = feature_importances.sort_values(ascending=False)

Model Evaluation

The performance of a Random Forest Classifier (RFC) model. When assessing a machine learning model, we consider various metrics to understand its effectiveness. Here are some key aspects of evaluating an RFC model:
1.	Accuracy:
-	Accuracy measures the proportion of correctly predicted instances out of the total instances in the test set.
-	It's a straightforward metric and is calculated as: [ \text{Accuracy} = \frac{\text{Number of Correct Predictions}}{\text{Total Number of Predictions}} ]
-	For example, if the RFC predicts 88% of instances correctly, the accuracy would be 0.884.
2.	Confusion Matrix:
-	A confusion matrix provides a detailed breakdown of the model's predictions.
-	It consists of four components:
 - True Positives (TP): Instances correctly predicted as positive.
  -	True Negatives (TN): Instances correctly predicted as negative.
  -	False Positives (FP): Instances incorrectly predicted as positive.
  -False Negatives (FN): Instances incorrectly predicted as negative.
-	The confusion matrix helps assess the trade-offs between precision and recall.
3.	Precision, Recall, and F1-Score:
-	These metrics are useful when dealing with imbalanced datasets or when certain classes are more critical than others.
-	Precision measures the proportion of true positive predictions among all positive predictions: [ \text{Precision} = \frac{TP}{TP + FP} ]
-	Recall (Sensitivity) measures the proportion of true positive predictions among all actual positive instances: [ \text{Recall} = \frac{TP}{TP + FN} ]
-	F1-score combines precision and recall into a single metric: [ F1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}} ]
4.	Receiver Operating Characteristic (ROC) Curve:
-	The ROC curve visualizes the trade-off between true positive rate (TPR) and false positive rate (FPR) at different classification thresholds.
-	The area under the ROC curve (AUC-ROC) quantifies the model's ability to distinguish between positive and negative classes. Higher AUC indicates better performance.
5.	Feature Importance:
-	RFC provides feature importance scores, indicating which features contribute most to the model's predictions.
-	Understanding feature importance helps identify critical variables and potential areas for improvement.
Remember that the choice of evaluation metrics depends on the specific problem and business context. It's essential to consider the trade-offs and select the most relevant metrics for your use case.



In [ ]:
# Model Evaluation
print("Accuracy:",accuracy_score(y_test,y_pred))
print("AUC-ROC",roc_auc_score(y_test,y_pred))

Accuracy: 0.9996324373994946
AUC-ROC 0.6739015510747396


In [ ]:
print('Confusion matrix:\n',confusion_matrix(y_test,y_pred))

Confusion matrix:
 [[28453     1]
 [   14    11]]


In [ ]:
print("Classification Report:\n",classification_report(y_test,y_pred))

Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     28454
         1.0       0.92      0.44      0.59        25

    accuracy                           1.00     28479
   macro avg       0.96      0.72      0.80     28479
weighted avg       1.00      1.00      1.00     28479



In [ ]:
print("Key Factor:",key_factors)

Key Factor: amount            0.278710
oldbalanceOrg     0.233480
newbalanceDest    0.223152
oldbalanceDest    0.194849
type              0.038750
newbalanceOrig    0.031060
dtype: float64
